In [178]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

In [8]:
data = pd.read_csv("hand-DS1.csv")
data

,min(X),min(Y),min(Z),max(X),max(Y),max(Z),mean(X),mean(Y),mean(Z),variance(X),...,energy(Y),energy(Z),dominantFr(X),dominantFrMag(X),dominantFr(Y),dominantFrMag(Y),dominantFr(Z),dominantFrMag(Z),user,8activity
0,-24.4424,-0.71655,-2.1629,0.05894,11.3562,8.9418,-9.087213,4.503875,2.414245,20.195223,...,23228.0,10389.0,1.562500,2492.956065,0.976562,120.122633,0.781250,266.657657,1.0,7.0
1,-24.4424,-0.71655,-1.4588,0.05894,11.3562,8.9418,-9.428112,4.355257,2.547715,20.750488,...,21850.0,10438.0,1.562500,2592.162870,1.562500,180.415276,1.367188,492.930505,1.0,7.0
2,-23.0289,-0.89687,-1.6693,-2.60030,8.4294,9.7131,-9.860712,4.129774,2.350786,20.099184,...,19850.0,8924.0,1.757812,4113.171474,1.757812,221.254022,1.757812,384.413869,1.0,7.0
3,-28.1650,-3.46070,-3.0401,-2.23780,11.0376,9.7131,-9.233821,4.152760,2.104738,23.455789,...,21143.0,9051.0,1.757812,2412.836035,0.976562,193.990278,0.781250,313.792051,1.0,7.0
4,-28.1650,-3.46070,-3.0401,-2.23780,11.0376,8.0647,-9.267435,4.206753,2.154653,21.942572,...,21137.0,8686.0,1.171875,1632.104209,1.757812,184.209968,1.171875,519.804950,1.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2229,-22.8838,-7.76560,-4.0637,3.55810,10.3989,14.8815,-6.581384,2.926745,6.708760,8.596861,...,14720.0,49413.0,7.031250,127.638843,19.726562,93.050052,22.265625,103.337972,8.0,6.0
2230,-16.1287,-4.58780,1.9891,-0.66287,9.5765,11.4903,-7.064428,2.826238,6.539265,6.109019,...,11978.0,44766.0,12.304688,83.727979,20.117188,82.295851,22.656250,29.621528,8.0,6.0
2231,-14.8416,-5.35010,1.9891,-1.16180,9.7720,12.0811,-6.932444,2.528950,6.735503,4.576157,...,9605.0,47387.0,0.000000,132.933505,19.726562,42.541833,0.000000,48.366647,8.0,6.0
2232,-15.9734,-5.35010,2.8923,0.19225,9.7720,12.0811,-6.416030,2.143886,7.089419,4.125772,...,8341.0,52149.0,0.976562,47.912812,14.257812,30.997972,0.976562,50.852010,8.0,6.0


In [159]:
subset = data[["min(X)", "min(Y)","min(Z)", "max(Y)", "max(Z)", "mean(X)", "variance(Z)",
               "std(X)","std(Y)","std(Z)", "median(X)", "percentile25(X)", "percentile75(X)",
               "percentile25(Z)","zerocross(X)", "zerocross(Z)","energy(X)","dominantFr(X)"]]
target = data[["8activity"]]
subset


,min(X),min(Y),min(Z),max(Y),max(Z),mean(X),variance(Z),std(X),std(Y),std(Z),median(X),percentile25(X),percentile75(X),percentile25(Z),zerocross(X),zerocross(Z),energy(X),dominantFr(X)
0,-24.4424,-0.71655,-2.1629,11.3562,8.9418,-9.087213,4.565075,4.493910,1.716533,2.136604,-8.40620,-12.61060,-5.91400,0.809745,32.0,32.0,102752.0,1.562500
1,-24.4424,-0.71655,-1.4588,11.3562,8.9418,-9.428112,3.951176,4.555270,1.698354,1.987756,-8.65635,-12.94840,-6.01420,0.890180,34.0,36.0,109619.0,1.562500
2,-23.0289,-0.89687,-1.6693,8.4294,9.7131,-9.860712,3.401346,4.483211,1.672559,1.844274,-8.83675,-13.59365,-6.06875,0.830155,35.0,32.0,117313.0,1.757812
3,-28.1650,-3.46070,-3.0401,11.0376,9.7131,-9.233821,4.626076,4.843118,1.975162,2.150832,-8.09370,-12.37470,-5.54065,0.429910,40.0,30.0,108696.0,1.757812
4,-28.1650,-3.46070,-3.0401,11.0376,8.0647,-9.267435,4.047756,4.684290,1.855651,2.011904,-8.38200,-12.71580,-5.33810,0.554130,34.0,32.0,107806.0,1.171875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2229,-22.8838,-7.76560,-4.0637,10.3989,14.8815,-6.581384,4.410102,2.932040,2.482026,2.100024,-6.29570,-8.09875,-4.77800,5.589500,293.0,350.0,51903.0,7.031250
2230,-16.1287,-4.58780,1.9891,9.5765,11.4903,-7.064428,2.005521,2.471643,1.998525,1.416164,-6.93235,-8.62525,-5.36400,5.616050,272.0,376.0,56009.0,12.304688
2231,-14.8416,-5.35010,1.9891,9.7720,12.0811,-6.932444,2.022523,2.139196,1.792480,1.422154,-6.79225,-8.15580,-5.46860,5.860100,249.0,298.0,52630.0,0.000000
2232,-15.9734,-5.35010,2.8923,9.7720,12.0811,-6.416030,1.890778,2.031200,1.936117,1.375055,-6.30010,-7.59650,-4.98915,6.216750,216.0,244.0,45287.0,0.976562


In [160]:
kfold = KFold(n_splits = 10)
cart = DecisionTreeClassifier()
num_trees = 20
baseTreeModel = BaggingClassifier(base_estimator = cart, n_estimators = num_trees)
baseTreeModel.fit(subset, target)
result = baseTreeModel.predict(subset)
resultsBaseTree = cross_val_score(baseTreeModel, subset, target, cv=kfold, scoring='f1_weighted')
print(resultsBaseTree.mean())


/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array 

0.7792686511632321


In [161]:
randomForestModel = RandomForestClassifier(max_features='sqrt')
resultsForest = cross_val_score(randomForestModel, subset, target, cv=kfold, scoring='f1_weighted')
print(resultsForest.mean())

/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn

0.7963323287065719


In [143]:
adaBoostModel = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators=20)
resultsAdaModel = cross_val_score(adaBoostModel, subset, target, cv=kfold,scoring='f1_weighted')
adaBoostModel.fit(subset, target)
print(resultsAdaModel.mean())

/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array 

0.7498440151447973


/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array 

In [177]:
kNeighborsModel = KNeighborsClassifier(n_neighbors = 7)
kNeighborsModel.fit(subset, target)
resultsKNeighborsModel = cross_val_score(kNeighborsModel, subset, target, cv=kfold,scoring='f1_weighted')
print(resultsKNeighborsModel.mean())
print(resultsKNeighborsModel)

/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y

0.4434093610860441
[0.36917565 0.35860572 0.52655675 0.43907681 0.43346125 0.29058744
 0.31589613 0.62833039 0.43452719 0.63787628]


In [172]:
svmModel = svm.SVC(probability=True)
svmModel.fit(subset, target)
resultsSvmModel = cross_val_score(svmModel, subset, target, cv=kfold,scoring='f1_weighted')
print(resultsSvmModel.mean())
print(resultsSvmModel)

/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array 

0.45809584819001625
[0.41376586 0.34310009 0.42331032 0.55556421 0.51979693 0.25803901
 0.39213017 0.35461524 0.57576115 0.74487549]


In [173]:
annModel = MLPClassifier(hidden_layer_sizes = (18,50,8), max_iter = 250)
annModel.fit(subset, target)
resultsAnnModel = cross_val_score(annModel, subset, target, cv=kfold,scoring='f1_weighted')
print(resultsAnnModel.mean())
print(resultsAnnModel)

/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array 

0.12985183178544651
[0.19872584 0.13392857 0.04078601 0.13125675 0.11070827 0.23755348
 0.15227896 0.06947573 0.05710694 0.16669777]


In [135]:
annModel
svmModel
kNeighborsModel
adaBoostModel

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=20)

In [215]:
#Weighted majority voting

pred1 = annModel.predict(subset)
pred2 = svmModel.predict(subset)
pred3 = kNeighborsModel.predict(subset)
pred4 = adaBoostModel.predict(subset)

finalpred=(pred1*0.25+pred2*0.25+pred3*0.25+pred4*0.25)
final = pd.DataFrame(finalpred)

,0
0,5.50
1,4.75
2,4.75
3,4.75
4,4.75
...,...
2229,4.25
2230,3.50
2231,3.00
2232,4.25


In [200]:
models = list()
models.append(annModel)
models.append(svmModel)
models.append(kNeighborsModel)
models.append(adaBoostModel)
ensemble = VotingClassifier(estimators=models, voting='soft')



/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_voting.py", line 292, in fit
    return super().fit(X, transformed_y, sample_weight)
  File "/Users/coltonproctor/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemb